In [ ]:
# %% [markdown]
# # 🧹 Limpieza y Procesamiento de Datos
# ## Análisis del Mercado Laboral de Data Science en España
# 
# **Objetivo:** Limpiar y preparar los datos crudos para análisis

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import json
from datetime import datetime

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %%
# Cargar los datos más recientes
data_dir = Path("../data/raw")
latest_file = sorted(data_dir.glob("jobs_data_*.csv"))[-1]
print(f"📂 Cargando: {latest_file.name}")

df_raw = pd.read_csv(latest_file)
print(f"✅ {len(df_raw)} ofertas cargadas")
df_raw.head()

# %% [markdown]
# ## 1️⃣ Inspección Inicial

# %%
# Dimensiones
print(f"📊 Dimensiones: {df_raw.shape[0]} filas × {df_raw.shape[1]} columnas\n")

# Info general
df_raw.info()

# %%
# Valores nulos
print("🔍 Valores nulos por columna:\n")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({
    'Nulos': missing,
    'Porcentaje': missing_pct
})
print(missing_df[missing_df['Nulos'] > 0].sort_values('Nulos', ascending=False))

# %%
# Duplicados
duplicates = df_raw.duplicated(subset=['id']).sum()
print(f"🔁 Ofertas duplicadas por ID: {duplicates}")

# %% [markdown]
# ## 2️⃣ Limpieza de Datos

# %%
# Crear copia para trabajar
df = df_raw.copy()

# %%
# 2.1 Eliminar duplicados
df = df.drop_duplicates(subset=['id'], keep='first')
print(f"✅ Duplicados eliminados: {len(df_raw) - len(df)}")
print(f"📊 Ofertas únicas: {len(df)}")

# %%
# 2.2 Limpiar y estandarizar ubicaciones
def clean_location(location):
    """Limpia y estandariza nombres de ubicaciones"""
    if pd.isna(location):
        return 'Desconocido'
    
    location = str(location).strip()
    
    # Extraer ciudad principal (antes de la coma)
    if ',' in location:
        location = location.split(',')[0].strip()
    
    # Estandarizar nombres comunes
    replacements = {
        'Donostia': 'San Sebastián',
        'Donostia-San Sebastián': 'San Sebastián',
        'A Coruña': 'La Coruña',
        'Zaragoza': 'Zaragoza',
        'Vizcaya': 'Bilbao',
        'Guipúzcoa': 'San Sebastián'
    }
    
    for old, new in replacements.items():
        if old in location:
            location = new
    
    return location

df['city'] = df['location_display'].apply(clean_location)
print(f"✅ Ubicaciones procesadas")
print(f"📍 Ciudades únicas: {df['city'].nunique()}")

# %%
# Top 10 ciudades
print("\n🏙️ Top 10 ciudades con más ofertas:")
print(df['city'].value_counts().head(10))

# %%
# 2.3 Clasificar nivel de experiencia desde el título
def extract_seniority(title):
    """Extrae el nivel de experiencia del título"""
    if pd.isna(title):
        return 'No especificado'
    
    title_lower = str(title).lower()
    
    if any(word in title_lower for word in ['junior', 'jr', 'trainee', 'graduate', 'entry']):
        return 'Junior'
    elif any(word in title_lower for word in ['senior', 'sr', 'lead', 'principal', 'staff']):
        return 'Senior'
    elif any(word in title_lower for word in ['manager', 'head', 'director', 'chief']):
        return 'Manager'
    else:
        return 'Mid-Level'

df['seniority'] = df['title'].apply(extract_seniority)
print("\n📊 Distribución de niveles de experiencia:")
print(df['seniority'].value_counts())

# %%
# 2.4 Categorizar tipos de roles
def categorize_role(title):
    """Categoriza el tipo de rol de data"""
    if pd.isna(title):
        return 'Otros'
    
    title_lower = str(title).lower()
    
    if 'scientist' in title_lower:
        return 'Data Scientist'
    elif 'analyst' in title_lower or 'analytics' in title_lower:
        return 'Data Analyst'
    elif 'engineer' in title_lower and ('data' in title_lower or 'ml' in title_lower):
        return 'Data/ML Engineer'
    elif 'machine learning' in title_lower or 'ml ' in title_lower:
        return 'Machine Learning'
    elif 'business intelligence' in title_lower or 'bi ' in title_lower:
        return 'Business Intelligence'
    elif 'ai' in title_lower or 'artificial intelligence' in title_lower:
        return 'AI Specialist'
    else:
        return 'Otros'

df['role_category'] = df['title'].apply(categorize_role)
print("\n💼 Distribución de categorías de roles:")
print(df['role_category'].value_counts())

# %%
# 2.5 Procesar salarios
# Convertir a numérico
df['salary_min'] = pd.to_numeric(df['salary_min'], errors='coerce')
df['salary_max'] = pd.to_numeric(df['salary_max'], errors='coerce')

# Calcular salario promedio
df['salary_avg'] = (df['salary_min'] + df['salary_max']) / 2

# Filtrar salarios realistas (entre 18K y 150K)
df.loc[(df['salary_avg'] < 18000) | (df['salary_avg'] > 150000), 'salary_avg'] = np.nan

print(f"\n💰 Ofertas con salario válido: {df['salary_avg'].notna().sum()} ({df['salary_avg'].notna().sum()/len(df)*100:.1f}%)")
print(f"💶 Salario promedio: {df['salary_avg'].mean():,.0f}€")
print(f"💶 Mediana salarial: {df['salary_avg'].median():,.0f}€")

# %%
# 2.6 Procesar fechas
df['created'] = pd.to_datetime(df['created'], errors='coerce')
df['collected_at'] = pd.to_datetime(df['collected_at'], errors='coerce')

# Extraer componentes de fecha
df['created_year'] = df['created'].dt.year
df['created_month'] = df['created'].dt.month
df['created_week'] = df['created'].dt.isocalendar().week

print("\n📅 Rango de fechas de publicación:")
print(f"Desde: {df['created'].min()}")
print(f"Hasta: {df['created'].max()}")

# %%
# 2.7 Limpiar nombres de empresas
def clean_company_name(company):
    """Limpia nombres de empresas"""
    if pd.isna(company):
        return 'No especificada'
    
    company = str(company).strip()
    
    # Eliminar sufijos comunes
    suffixes = [' S.L.', ' S.A.', ' SL', ' SA', ' Ltd', ' Inc', ' Corp']
    for suffix in suffixes:
        company = company.replace(suffix, '')
    
    return company.strip()

df['company_clean'] = df['company_name'].apply(clean_company_name)
print(f"\n🏢 Empresas únicas: {df['company_clean'].nunique()}")

# %% [markdown]
# ## 3️⃣ Extracción de Skills

# %%
# Lista de skills a buscar (ampliada)
SKILLS = {
    # Lenguajes
    'Python': r'\bpython\b',
    'R': r'\b r\b|\br programming\b',
    'SQL': r'\bsql\b',
    'Java': r'\bjava\b',
    'Scala': r'\bscala\b',
    
    # ML/DL
    'TensorFlow': r'\btensorflow\b',
    'PyTorch': r'\bpytorch\b',
    'Keras': r'\bkeras\b',
    'scikit-learn': r'\bscikit.learn\b|\bsklearn\b',
    'XGBoost': r'\bxgboost\b',
    
    # Big Data
    'Spark': r'\bspark\b',
    'Hadoop': r'\bhadoop\b',
    'Kafka': r'\bkafka\b',
    'Airflow': r'\bairflow\b',
    
    # Databases
    'PostgreSQL': r'\bpostgresql\b|\bpostgres\b',
    'MySQL': r'\bmysql\b',
    'MongoDB': r'\bmongodb\b',
    'Elasticsearch': r'\belasticsearch\b',
    
    # Cloud
    'AWS': r'\baws\b',
    'Azure': r'\bazure\b',
    'GCP': r'\bgcp\b|\bgoogle cloud\b',
    
    # BI Tools
    'Tableau': r'\btableau\b',
    'Power BI': r'\bpower bi\b|\bpowerbi\b',
    'Looker': r'\blooker\b',
    
    # Otros
    'Docker': r'\bdocker\b',
    'Kubernetes': r'\bkubernetes\b|\bk8s\b',
    'Git': r'\bgit\b',
    'Pandas': r'\bpandas\b',
    'NumPy': r'\bnumpy\b',
}

# %%
# Extraer skills de las descripciones
def extract_skills(description):
    """Extrae skills mencionadas en la descripción"""
    if pd.isna(description):
        return []
    
    description_lower = str(description).lower()
    found_skills = []
    
    for skill, pattern in SKILLS.items():
        if re.search(pattern, description_lower, re.IGNORECASE):
            found_skills.append(skill)
    
    return found_skills

print("🔍 Extrayendo skills de las descripciones...")
df['skills'] = df['description'].apply(extract_skills)
df['num_skills'] = df['skills'].apply(len)

print(f"✅ Skills extraídas")
print(f"📊 Promedio de skills por oferta: {df['num_skills'].mean():.1f}")

# %%
# Skills más demandadas
from collections import Counter

all_skills = [skill for skills_list in df['skills'] for skill in skills_list]
skill_counts = Counter(all_skills)

print("\n🔥 Top 15 skills más demandadas:")
for skill, count in skill_counts.most_common(15):
    percentage = (count / len(df)) * 100
    print(f"{skill:20} {count:4} ofertas ({percentage:5.1f}%)")

# %% [markdown]
# ## 4️⃣ Detección de IA/ML

# %%
# Detectar menciones de IA/ML/GPT
AI_KEYWORDS = [
    'artificial intelligence', 'ai', 'machine learning', 'ml', 
    'deep learning', 'neural network', 'llm', 'gpt', 'chatgpt',
    'generative ai', 'computer vision', 'nlp', 'natural language'
]

def has_ai_keywords(description):
    """Detecta si la oferta menciona IA/ML"""
    if pd.isna(description):
        return False
    
    description_lower = str(description).lower()
    return any(keyword in description_lower for keyword in AI_KEYWORDS)

df['is_ai_related'] = df['description'].apply(has_ai_keywords)
ai_jobs = df['is_ai_related'].sum()

print(f"🤖 Ofertas relacionadas con IA/ML: {ai_jobs} ({ai_jobs/len(df)*100:.1f}%)")

# %% [markdown]
# ## 5️⃣ Guardar Datos Limpios

# %%
# Seleccionar columnas finales
columns_to_keep = [
    'id', 'title', 'company_clean', 'city', 'role_category', 'seniority',
    'salary_min', 'salary_max', 'salary_avg', 'description',
    'created', 'skills', 'num_skills', 'is_ai_related',
    'redirect_url', 'contract_type', 'contract_time'
]

df_clean = df[columns_to_keep].copy()

# Renombrar columnas
df_clean = df_clean.rename(columns={
    'company_clean': 'company',
    'redirect_url': 'url'
})

# Guardar
output_path = Path("../data/processed/jobs_cleaned.csv")
df_clean.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n💾 Datos limpios guardados: {output_path}")
print(f"📊 Total: {len(df_clean)} ofertas")
print(f"📝 Columnas: {len(df_clean.columns)}")

# %%
# Resumen final
print("\n" + "="*70)
print("✅ LIMPIEZA COMPLETADA".center(70))
print("="*70)
print(f"\n📊 Dataset final:")
print(f"   • {len(df_clean)} ofertas únicas")
print(f"   • {df_clean['city'].nunique()} ciudades")
print(f"   • {df_clean['company'].nunique()} empresas")
print(f"   • {df_clean['salary_avg'].notna().sum()} ofertas con salario")
print(f"   • {df_clean['is_ai_related'].sum()} ofertas de IA/ML")
print(f"   • Promedio: {df_clean['num_skills'].mean():.1f} skills por oferta")

print("\n🎯 Siguiente paso: Análisis exploratorio detallado (notebook 03)")

ModuleNotFoundError: No module named 'pandas'